<h1><center><font color='orange'>Credit Card Fraud Detection</center></h1>

---

<h4><center><font color='lightblue'>Author - Satya Dileep Penmetsa</center></h4>


<img src = 'https://images.ctfassets.net/appqvz9wcil2/2jRwdiHx3CVQhzaoc9cr18/1474e4243a3f65b8bf1372566009e6d0/19_06_04_Blog-Chargeback__1_.gif'>

In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import warnings

# Disable warnings emitted by warnings.warn calls from different packages
# matplotlib can show a warning with tight_layout that can be safely ignored

warnings.filterwarnings('ignore')

# Matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
from matplotlib.colorbar import ColorbarBase

# Scipy
from scipy import interpolate
from scipy import spatial
from scipy import stats
from scipy.cluster import hierarchy
# Others
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import pickle
from math import modf

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn import metrics
import xgboost as xgb
from sklearn.linear_model import SGDClassifier


sns.set(style='white', context='notebook', palette='deep')
from sklearn.model_selection import train_test_split, KFold, cross_validate

#Models
import warnings
warnings.filterwarnings("ignore")


from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import StratifiedShuffleSplit, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve, precision_recall_fscore_support, roc_auc_score
from sklearn.inspection import plot_partial_dependence

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier  
from sklearn.exceptions import ConvergenceWarning

pd.options.display.float_format = '{:.5f}'.format
import numpy as np
import matplotlib.pyplot as plt
plt.style.context('seaborn-talk')
plt.style.use('fivethirtyeight')
params = {'legend.fontsize': '16',
          'figure.figsize': (15, 5),
         'axes.labelsize': '20',
         'axes.titlesize':'30',
         'xtick.labelsize':'18',
         'ytick.labelsize':'18'}
plt.rcParams.update(params)

plt.rcParams['text.color'] = '#A04000'
plt.rcParams['xtick.color'] = '#283747'
plt.rcParams['ytick.color'] = '#808000'
plt.rcParams['axes.labelcolor'] = '#283747'

# Seaborn style (figures)
sns.set(context='notebook', style='whitegrid')
sns.set_style('ticks', {'xtick.direction':'in', 'ytick.direction':'in'})

from ipywidgets import interact

In [4]:
data_df = pd.read_csv("data/creditcard.csv")

<h3><center>Explore Data</center></h3>

In [5]:
data_df.head(5).style.background_gradient(cmap ='Pastel1').set_properties(**{'font-size': '9px'}) 

In [6]:
@interact

def create_scatter(Feature1=data_df.columns, Feature2=data_df.columns):
    with plt.style.context("ggplot"):
        fig = plt.figure(figsize=(25,7))

        plt.scatter(x = data_df[Feature1],
                    y = data_df[Feature2],
                    c=data_df["Class"],
                    s=20
                   )

        plt.xlabel(Feature1.capitalize())
        plt.ylabel(Feature2.capitalize())

        plt.title("%s vs %s"%(Feature1.capitalize(), Feature2.capitalize()))
        
#widgets.interact(create_scatter, Feature1=data_df.columns, Feature2=data_df.columns);

interactive(children=(Dropdown(description='Feature1', options=('Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V…

<center> There is no missing data in the entire dataset. But, the data was imbalanced. Let's check data unbalance with respect with target value, i.e. Class.</center>

In [7]:
temp = data_df["Class"].value_counts()
df = pd.DataFrame({'Class': temp.index,'values': temp.values})

trace = go.Bar(
    x = df['Class'],y = df['values'],
    name="Credit Card Fraud Class - data unbalance (Not fraud = 0, Fraud = 1)",
    #marker=dict(color="Blue"),
    text=df['values']
)
data = [trace]
layout = dict(title = 'Credit Card Fraud Class - data unbalance (Not fraud = 0, Fraud = 1)',
          xaxis = dict(title = 'Class', showticklabels=True), 
          yaxis = dict(title = 'Number of transactions'),
          hovermode = 'closest',width=1475
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='class')

<center> Only 492 (or 0.172%) of transaction are fraudulent. That means the data is highly unbalanced with respect with target variable Class. </center>

# <a id="5"><center>Data exploration</center></a>

<h3><center>Transactions in time</center></h3>

In [ ]:
class_0 = data_df.loc[data_df['Class'] == 0]["Time"]
class_1 = data_df.loc[data_df['Class'] == 1]["Time"]
hist_data = [class_0, class_1]
group_labels = ['Not Fraud', 'Fraud']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig['layout'].update(title='Credit Card Transactions Time Density Plot', width = 1475, xaxis=dict(title='Time [s]'))
iplot(fig, filename='dist_only')

<center>Fraudulent transactions have a distribution more even than valid transactions - are equaly distributed in time, including the low real transaction times, during night in Europe timezone.</center>

<h3><center>Transactions amount</center></h3>

In [ ]:
params = {'legend.fontsize': '20',
          'figure.figsize': (15, 5),
         'axes.labelsize': '18',
         'axes.titlesize':'30',
         'xtick.labelsize':'16',
         'ytick.labelsize':'16'}
plt.rcParams.update(params)

plt.rcParams['text.color'] = '#A04000'
plt.rcParams['xtick.color'] = '#800000'
plt.rcParams['ytick.color'] = '#808000'
plt.rcParams['axes.labelcolor'] = '#283747'

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(25,6))
s = sns.boxplot(ax = ax1, x="Class", y="Amount", hue="Class",data=data_df, palette="PRGn",showfliers=True)
s = sns.boxplot(ax = ax2, x="Class", y="Amount", hue="Class",data=data_df, palette="PRGn",showfliers=False)
plt.show();

In [ ]:
tmp = data_df[['Amount','Class']].copy()
class_0 = tmp.loc[tmp['Class'] == 0]['Amount']
class_1 = tmp.loc[tmp['Class'] == 1]['Amount']

<center>The real transaction have a larger mean value, larger Q1, smaller Q3 and Q4 and larger outliers; fraudulent transactions have a smaller Q1 and mean, larger Q4 and smaller outliers.</center>

<center>Let's plot the fraudulent transactions (amount) against time. The time is shown is seconds from the start of the time period (totaly 48h, over 2 days).<center>

In [ ]:
fraud = data_df.loc[data_df['Class'] == 1]

trace = go.Scatter(
    x = fraud['Time'],y = fraud['Amount'],
    name="Amount",
     marker=dict(
                color='rgb(238,23,11)',
                line=dict(
                    color='red',
                    width=1),
                opacity=0.5,
            ),
    text= fraud['Amount'],
    mode = "markers"
)
data = [trace]
layout = dict(title = 'Amount of fraudulent transactions',
          xaxis = dict(title = 'Time [s]', showticklabels=True), 
          yaxis = dict(title = 'Amount'),
          hovermode='closest',
              width = 1475
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='fraud-amount')

# <a id="5"><center>Features correlation</center></a>

In [ ]:
plt.figure(figsize = (27,7))
plt.title('Credit Card Transactions features correlation plot (Pearson)')
corr = data_df.corr()
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,cmap="Reds")
plt.show()

<center>As expected, there is no notable correlation between features **V1**-**V28**. </center>

# <a id="2"><center>Features density plot</center></a>

In [ ]:
var = data_df.columns.values

i = 0
t0 = data_df.loc[data_df['Class'] == 0]
t1 = data_df.loc[data_df['Class'] == 1]

sns.set_style('whitegrid')
fig, ax = plt.subplots(8,4,figsize=(25,18))

for feature in var:
    i += 1
    plt.subplot(8,4,i)
    sns.kdeplot(t0[feature], bw=0.5,label="Class = 0")
    sns.kdeplot(t1[feature], bw=0.5,label="Class = 1")
    plt.xlabel(feature, fontsize=12)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=12)

plt.show();

For some of the features we can observe a good selectivity in terms of distribution for the two values of **Class**: **V4**, **V11** have clearly separated distributions for **Class** values 0 and 1, **V12**, **V14**, **V18** are partially separated, **V1**, **V2**, **V3**, **V10** have a quite distinct profile, whilst **V25**, **V26**, **V28** have similar profiles for the two values of **Class**. 


In general, with just few exceptions (**Time** and **Amount**), the features distribution for legitimate transactions (values of **Class = 0**)  is centered around 0, sometime with a long queue at one of the extremities. In the same time, the fraudulent transactions (values of **Class = 1**) have a skewed (asymmetric) distribution.

In [ ]:
data_df = data_df[data_df['Class']!='NA']
vals = data_df['Class'].value_counts()

### <center><font color='orange'>Before Applying Over Sampling for Balancing the Classes</font></center>



In [ ]:
# Plotting attrition of employees
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(26,5))

ax1 = data_df['Class'].value_counts().plot.pie(ax = ax1, shadow=True);
ax1.set(title = 'Number of Transactions by class')

ax2 = data_df['Class'].value_counts().plot(kind="barh" ,ax =ax2)
ax2.set(title = 'Number of Transactions by class')
plt.show()

In [ ]:
final_df = data_df.copy()

In [ ]:
final_df.drop(['Time'] ,axis=1, inplace=True)

### <center><font color='green'>After Applying Over Sampling for Balancing the Classes</font></center>

In [ ]:

#Over-sampling: SMOTE
#SMOTE (Synthetic Minority Oversampling TEchnique) consists of synthesizing elements for the minority class, 
#based on those that already exist. It works randomly picking a point from the minority class and computing 
#the k-nearest neighbors for this point.The synthetic points are added between the chosen point and its neighbors.
#We'll use ratio='minority' to resample the minority class.
smote = SMOTE('minority')
finalDF = final_df.copy()
X = finalDF.drop(['Class'], axis=1)
Y = finalDF['Class']
X_sm, Y_sm = smote.fit_sample(X, Y)

In [ ]:
unique_elements, counts_elements = np.unique(Y_sm, return_counts=True)

In [ ]:
# Plotting attrition of employees
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(26,5))

ax1 = pd.value_counts(Y_sm).plot(kind = 'pie', ax = ax1, shadow=True);
ax1.set(title = 'Number of Transactions by class')

ax2 = pd.value_counts(Y_sm).plot(kind="barh" ,ax =ax2)
ax2.set(title = 'Number of Transactions by class')
plt.show()


## <center>Ensemble Modelling Results</center>

### <center>Comparison of Ensemble Models - Cross Validation</center>

In [ ]:
cv_res = pd.read_csv('imgs/CVscores.csv')
plt.style.use('bmh')
plt.figure(figsize=(25,5))
g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h")
g.set_xlabel("Mean Accuracy Score")
g = g.set_title("Cross validation scores")

### <center>Comparison of Ensemble Models - Train Test Split</center>

In [ ]:
import numpy as np
scoreDF = pd.read_csv('imgs/traintest_results.csv')
models = ["RandomForest", "AdaBoost", "ExtraTrees","GradientBoosting","XGboost"]

plt.style.use('fivethirtyeight')
def subcategorybar(X, vals, width=0.8):
    cols = ['Train Accuracy', 'Test Accuracy', 'Precision', 'Recall','F1 Score']
    n = len(vals)
    _X = np.arange(len(X))
    for i in range(n):
        plt.bar(_X - width/2. + i/float(n)*width, vals[i], width=width/float(n), align="edge")   
        
        
    plt.xticks(_X, X)
    
plt.figure(figsize = (25,5))
subcategorybar(models, [scoreDF['Train Accuracy'], scoreDF['Test Accuracy'], scoreDF['Precision'], scoreDF['Recall'], scoreDF['F1 Score']])
plt.ylim(0.98, 1.0)
cols = ['Train Accuracy', 'Test Accuracy', 'Precision', 'Recall','F1 Score']
plt.legend(cols, bbox_to_anchor=(1.01, 1.01),fancybox=False, shadow=False)
plt.xlabel('Model')
plt.title("Comparison of Models")
plt.show()

## <center>Ensemble on Traditional Models</center>

<center>1. Simple Averaging - Predict the class with the largest sum of votes from models</center>
<center>2. Soft Voting - Predict the class with the largest summed probability from models.</center>
<center>3. Hard Voting - Predict the class with the largest sum of votes from models</center>

In [ ]:
VotingEnsemblesDF = pd.read_csv('imgs/VotingEnsembles.csv')
models = ['Simple Averaging',
 'Weighted Averaging (Soft Voting)',
 'Weighted Averaging (Hard Voting)']
    
plt.figure(figsize = (25,5))
subcategorybar(models, [VotingEnsemblesDF['Train Accuracy'], VotingEnsemblesDF['Test Accuracy'], VotingEnsemblesDF['Precision'], VotingEnsemblesDF['Recall'], VotingEnsemblesDF['F1 Score']])
plt.ylim(0.9, 1.0)
cols = ['Train Accuracy', 'Test Accuracy', 'Precision', 'Recall','F1 Score']
plt.legend(cols, bbox_to_anchor=(1.01, 1.01), fancybox=False, shadow=False)
plt.xlabel('Model')
plt.title("Comparison of Models")
plt.show()

## <center>Stacking Based Ensembles</center>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
StackingEnsemblesDF = pd.read_excel('imgs/StackedEnsembles.xlsx')

models = ['Logistic Regression Classifier',
 'Linear Discriminant Analysis Classifier',
 'Stochastic Gradient Descent Classifier',
 'Decision Tree Classifier',
 'Random Forest Classifier',
 'Bagging Classifier',
 'XGBoost Classifier',
 'Adaboost Classifier',
 'Gradient Boosting Machine Classifier',
 'Extra Trees Classifier',
 'Stacked Ensemble Classifier',
 'Deep Learning Ensemble Classifier']
    
plt.figure(figsize = (25,5))
subcategorybar(models, [StackingEnsemblesDF['Accuracy Score (%)'], StackingEnsemblesDF['Precision'], StackingEnsemblesDF['Sensitivity'], StackingEnsemblesDF['Specificity'], 
                        StackingEnsemblesDF['F1 Score'], StackingEnsemblesDF['Receiver Operating Characteristics (ROC)'], StackingEnsemblesDF['Log Loss'], StackingEnsemblesDF['Matthews Correlation Coefficient (MCC)']])
plt.ylim(0.8, 1.0)
cols = ['Accuracy Score (%)',
        'Precision', 
        'Sensitivity',
        'Specificity',
        'F1 Score',
        'Receiver Operating Characteristics (ROC)',
        'Log Loss',
        'Matthews Correlation Coefficient (MCC)'
]
plt.xticks(rotation = 90)
plt.legend(cols, bbox_to_anchor=(1.01, 1.01),  fancybox=False, shadow=False)
plt.xlabel('Model')
plt.title("Comparison of Models")
plt.show()

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive
import seaborn as sns
# Set colormap equal to seaborns light green color palette
cm = sns.light_palette("green", as_cmap=True)
# Set table styles

# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '11px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#6d6d6d'),
  ('background-color', '#f7f7f9')
  ]

# Set CSS properties for td elements in dataframe
td_props = [
  ('font-size', '28px')
  ]

def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "20pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

styles = [
  magnify(),
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
  ]

    
items = sorted(StackingEnsemblesDF['Model Name'].unique().tolist())
 
def view(x=''):
    if x=='All': return display(StackingEnsemblesDF)
    return display(StackingEnsemblesDF[StackingEnsemblesDF['Model Name']==x].style
  .background_gradient(cmap=cm, subset=['Accuracy Score (%)'])
  .background_gradient(cmap='Blues')
  .highlight_max(color='lightgreen')
  .highlight_min(color='#lightgreen')
  .highlight_max(subset=['Accuracy Score (%)'])
  .set_caption('Selected Model')
  .format({'Accuracy Score (%)': "{:.2%}"})
  .hide_index()
  .set_table_styles(styles))

 
w = widgets.Select(options=items)
interactive(view, x=w)


## <center> ROC and Precision Recall Curves</center>

In [ ]:
@interact

def line_plot(columns = ['Model Name'], model = sorted(StackingEnsemblesDF['Model Name'].unique())):
    
    StackingEnsemblesDF[StackingEnsemblesDF['Model Name'] == model].plot(kind = 'bar', title = model, figsize = (17,6))
    plt.xlabel('Selected Model')
    plt.ylabel('Evaluation Scores')
    plt.ylim(0.0, 1.0)
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    plt.show()

## <center>Feature Importance  <a id='data-conc'></a></center>

- As we have seen, stacked ensemble of power machine learning algorithms resulted in higher performance than any individual machine learning model.
- We have also interpreted second best performing algo i.e., random forest algorithm
- The top most contribution features are shown in the feature importance plot below. 


<p align="center">
  <img src="imgs/merge.jpg" width = 2500>
</p>